In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils.data_utils import data_generators

In [94]:
# Data path
competition_name = "semi-conductor-image-classification-first"
data_dir = os.path.expanduser(
    f"~/.kaggle/competitions/{competition_name}")
    
# Input parameters
SIDE_LENGTH = 224  # default 224
IMAGE_WIDTH = IMAGE_HEIGHT = SIDE_LENGTH
image_size = (IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS = 1
input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)
num_classes = 2

# Data loaders
train_generator, validation_generator = data_generators(
    data_dir, target_size=image_size, batch_size=16, seed=42)

Found 24000 images belonging to 2 classes.
Found 6000 images belonging to 2 classes.


In [79]:
class_indices = train_generator.class_indices
print(f"class_indices: {train_generator.class_indices}")

class_indices: {'good_0': 0, 'bad_1': 1}


## Dataset statistics

In [108]:
num_train = len(train_generator.filenames)
num_val = len(validation_generator.filenames)
print("Train set size:")
print(num_train)
print("Validation set size:")
print(num_val)
all_files = set(train_generator.filenames + validation_generator.filenames)
print("Total size:")
print(len(all_files))

Train set size:
24000
Validation set size:
6000
Total size:
30000


In [107]:
def is_good_0(f):
    return 1 if f.startswith("good_0") else 0

21600

In [111]:
num_good_in_train = sum([is_good_0(_) for _ in train_generator.filenames])
num_bad_in_train = num_train - num_good_in_train

In [112]:
print(num_good_in_train, num_bad_in_train)

21600 2400


In [113]:
num_good_in_val = sum([is_good_0(_) for _ in validation_generator.filenames])
num_bad_in_val = num_val - num_good_in_val

In [114]:
print(num_good_in_val, num_bad_in_val)

5400 600


## Data visualization